In [1]:
import shap 
import torch 
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd 

import sys 
sys.path.append('../')
import src.models.utils as utils

/home/teddy/miniconda3/envs/grcp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torch.load('../output/exp1/0d5d60ef-b896-4557-8620-8522bcfca888/model.pt').cpu()

In [3]:
data = torch.load('../output/exp1/proc/Data.pt')

In [4]:
data.node_names

array(['DRUG__BRD-A00758722', 'DRUG__BRD-A00938334',
       'DRUG__BRD-A03249105', ..., 'RNA__Q9Y6Q9', 'RNA__Q9Y6R4',
       'RNA__Q9Y6W5'], dtype='<U19')

In [5]:
B = 100 

x = torch.randn((B, data.node_names.shape[0]))

x.size()

torch.Size([100, 3282])

In [21]:


ex = utils.node2edge(x, model.edge_index)
f = lambda ex: model.edge_update(x=torch.tensor(ex), x0=None, x_last=torch.zeros_like(torch.tensor(ex)).unsqueeze(-1), layer=0)[0].detach().numpy()
ey = f(ex.detach().numpy())
y = utils.edge2node(ey, model.edge_index, output_node_mask=model.output_node_mask)

TypeError: 'int' object is not callable

In [26]:
explainer = shap.Explainer(f, ex[:25, :].detach().numpy())
shap_values = explainer(ex[25:, :].detach().numpy())

ValueError: max_evals=500 is too low for the Permutation explainer, it must be at least 2 * num_features + 1 = 28711!

In [27]:
def extract_function(node, model, data): 
    '''
    extract a single node function from the GSNN model and return the neural network. 
    '''
    C = 3 #model.channels
    node_idx = list(data.node_names).index(node)

    # get first layer weights 
    src, dst = model.lin1.indices
    node_idxs = torch.arange(node_idx*C, node_idx*C + C)
    edge_mask = torch.isin(dst, node_idxs)

    lin1 = copy.deepcopy(model.lin1)
    
    W1 = model.lin1.values[edge_mask]
    B1 = model.lin1.bias[node_idxs] 

    input_names = data.node_names[src[edge_mask]]

    print(input_names)
    print(W1)
    print(B1)

    

In [28]:
extract_function(node='PROTEIN__P26045', model=model, data=data)

IndexError: index 12386 is out of bounds for axis 0 with size 3282

In [21]:
'PROTEIN__P26045' in data.node_names.tolist() 

True